In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os

import copy
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
import cv2
import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.applications.vgg16 import VGG16,preprocess_input


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('Path to your main Directory')

# Data Loader 
## Here i have written custom Loop to read images and labels

In [ ]:
import os
import numpy as np
from sklearn.utils import shuffle
from PIL import Image

W = 224
H = 224
dataset = "images"
label_to_class = {'Fractured': 0, 'Non_fractured': 1}

Images = []
Classes = []

class_to_label = {v: k for k, v in label_to_class.items()}
n_classes = len(label_to_class)

for folder in os.listdir(dataset):
    img_folder = os.path.join(dataset, folder)
    cls = label_to_class[folder]
    i = 0

    for img_name in os.listdir(img_folder):
        img_path = os.path.join(img_folder, img_name)
        try:
        # Use Pillow to open and resize the image
          img = Image.open(img_path)

          # Check if the image is grayscale
          if img.mode == 'L':
              img = img.convert('RGB')
          img = img.resize((W, H))
          # Convert to numpy array
          img = np.array(img)

        except Exception as e:
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, (W, H))
        Images.append(img)
        Classes.append(cls)


        if i == 800:
            break
        i = i + 1

Images = np.array(Images, dtype=np.float32)
Classes = np.array(Classes, dtype=np.float32)
Images, Classes = shuffle(Images, Classes, random_state=0)


In [ ]:
Images.shape, Classes.shape

((1518, 224, 224, 3), (1518,))

In [ ]:
Classes

array([1., 0., 0., ..., 1., 0., 0.], dtype=float32)

# Spliting The data into train and Test

In [ ]:
## split train / test

indices_train, indices_test = train_test_split(list(range(Images.shape[0])), train_size=0.8, test_size=0.2, shuffle=True)

x_train = Images[indices_train]
y_train = Classes[indices_train]
x_test = Images[indices_test]
y_test = Classes[indices_test]

x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1214, 224, 224, 3), (1214,), (304, 224, 224, 3), (304,))

In [ ]:
y_test

array([1., 1., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0.,
       1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0.,
       1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
       1., 0., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0.,
       0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 0.,
       0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1.,
       1., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
       0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 1., 1., 1., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1.,
       1., 0., 0., 1., 1.

In [ ]:
from keras.utils import to_categorical

In [ ]:
y_train = keras.utils.to_categorical(y_train, n_classes)
y_test = keras.utils.to_categorical(y_test, n_classes)

y_train.shape, y_test.shape

((1214, 2), (304, 2))

In [ ]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1214, 224, 224, 3), (1214, 2), (304, 224, 224, 3), (304, 2))

## Pretrained Transformer Model

In [ ]:
!pip install -U tensorflow-addons

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [ ]:
num_classes = 2
input_shape = (224,224, 3)

In [ ]:
!pip install vit_keras

In [ ]:
from vit_keras import vit

vit_model = vit.vit_b16(
        image_size = image_size,
        activation = 'softmax',
        pretrained = True,
        include_top = False,
        pretrained_top = False,
        classes = 2)

model = tf.keras.Sequential([
        vit_model,
        tf.keras.layers.Flatten(),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation = tfa.activations.gelu),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(64, activation = tfa.activations.gelu),
        tf.keras.layers.Dense(32, activation = tfa.activations.gelu),
        tf.keras.layers.Dense(2, 'softmax')
    ],
    name = 'vision_transformer')

In [ ]:
from matplotlib import pyplot
from sklearn.metrics import precision_score,recall_score,f1_score,accuracy_score,classification_report

### Model Training

In [1]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate = learning_rate,
                                 weight_decay = weight_decay),
    loss=keras.losses.CategoricalCrossentropy(),
    metrics=[keras.metrics.CategoricalAccuracy(name="accuracy")],
)

model.summary()


In [1]:

# Assign class weight in case of imbalance and set it in model.fit too
# class_weight_dict = {0: 0.3, 1: 0.2}  # class weights directly

history = model.fit(
    x_train, y_train, batch_size=2, epochs=4, validation_data=(x_test, y_test)
)

# evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {round(loss, 2)}")
print(f"Test accuracy: {round(accuracy * 100, 2)} %")


In [ ]:
y_pred=model.predict(x_test)

In [ ]:
y_pred=np.argmax(y_pred,axis=1)

In [ ]:
y_pred

In [ ]:
len(y_test),len(y_pred)

In [ ]:
y_testt=np.argmax(y_test, axis=1)

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_testt, y_pred)
cm

In [ ]:
plt.figure(figsize = (10,7))
sns.heatmap(cm, annot=True, fmt='g')
plt.show()

In [ ]:
classes = ['fractured','no fractured']
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred,target_names=classes))


In [ ]:
# evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {round(loss, 2)}")
print(f"Test accuracy: {round(accuracy * 100, 2)} %")